## Creating Bokeh Apps
- Basic App Structure
    - Create a plot
    - Create a widget
    - Connect widget & plots (callbacks)
    - Arrange all the plots and widgets together to layouts
    - use `curdoc().add_root()` to add the layouts to the current document

### Serving Bokeh Apps
- bokeh serve --show app.py
- bokeh serve --show appdir/

#### Will not necessarily be showing optimal way to fun things but in a way that is most understandable


* ** The below code will not run but we will walk through each step to understand what is going on **

In [ ]:
# "current document" where all the the plots and layouts are held
from bokeh.io import curdoc 

from bokeh.plotting import figure
from bokeh.models import ColumnDataSource

import pandas as pd

# read the dataset
service_requests = pd.read_csv('../datasets/service-requests.csv', index_col=0)

# convert it to a ColumnDataSource
sr_cds = ColumnDataSource(service_requests)

# create the blank figure
p = figure()

# create circle glyphs with latitude and longitude coordinates as x and y
p.circle('LATITUDE', 'LONGITUDE', source=sr_cds)

# add the plot to the current document
curdoc().add_root(p)

<img src="../images/311-requests/101.png", height=400, width=400>

* Now we'll build a drop down menu

* The 'CASE_STATUS' column within the 311 service requests dataset has two options. 'CLOSED' and 'OPEN'.
* Screenshot of how you could find this out for any particular column of data

<img src="../images/311-requests/case-status-options.png">

## Adding a dropdown menu

In [ ]:
# "current document" where all the the plots and layouts are held
from bokeh.io import curdoc 

from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, Select
from bokeh.layouts import row

import pandas as pd

# read the dataset
service_requests = pd.read_csv('../datasets/service-requests.csv', index_col=0)

# convert it to a ColumnDataSource
sr_cds = ColumnDataSource(service_requests)

# create the blank figure
p = figure()

# create circle glyphs with latitude and longitude coordinates as x and y
p.circle('LATITUDE', 'LONGITUDE', source=sr_cds)

# create a menu widget to be added to the curdoc
menu = Select(options=['Closed', 'Open'], value='Closed', title='Case Status')

# create a layout with one row
layout = row(p, menu)

# add the layout to the current document
curdoc().add_root(layout)

<img src="../images/311-requests/102.png", height=400, width=400>

## Connecting the Menu with the plot

In [ ]:
# "current document" where all the the plots and layouts are held
from bokeh.io import curdoc 

from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, Select
from bokeh.layouts import row

import pandas as pd

# read the dataset
service_requests = pd.read_csv('../datasets/service-requests.csv', index_col=0)

# filter the datasets by closed and open
sr_closed = service_requests[service_requests['CASE_STATUS'] == 'Closed']
sr_open = service_requests[service_requests['CASE_STATUS'] == 'Open']

# convert it to a ColumnDataSource
sr_cds = ColumnDataSource(service_requests)
sr_cds_closed = ColumnDataSource(sr_closed)
sr_cds_open = ColumnDataSource(sr_open)

# create the blank figure
p = figure()

# create circle glyphs with latitude and longitude coordinates as x and y
p.circle('LATITUDE', 'LONGITUDE', source=sr_cds)

# create a menu widget to be added to the curdoc
menu = Select(options=['Closed', 'Open'], value='Closed', title='Case Status')

# create a function to connect the menu with the data source
def update_plot(attr, old, new):
    if menu.value == 'Closed':
        # take the original CDS and replace the values from the LATITUDE and LONGITUDE keys
        # with the values from the filtered CDS
        sr_cds.data = {
        'LATITUDE':sr_cds_closed.data['LATITUDE'],
        'LONGITUDE':sr_cds_closed.data['LONGITUDE'],
        }
    else:
        sr_cds.data = {
        'LATITUDE':sr_cds_open.data['LATITUDE'],
        'LONGITUDE':sr_cds_open.data['LONGITUDE'],
        }


menu.on_change('value', update_plot)

# create a layout with one row
layout = row(p, menu)

# add the layout to the current document
curdoc().add_root(layout)

<img src="../images/311-requests/103.png", height=400, width=400>

## Adding a map to the data

In [ ]:
# "current document" where all the the plots and layouts are held
from bokeh.io import curdoc 

from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, Select
from bokeh.layouts import row
from bokeh.tile_providers import CARTODBPOSITRON

import pandas as pd

# read the dataset
service_requests = pd.read_csv('../datasets/service-requests.csv', index_col=0)

# filter the datasets by closed and open
sr_closed = service_requests[service_requests['CASE_STATUS'] == 'Closed']
sr_open = service_requests[service_requests['CASE_STATUS'] == 'Open']

# convert it to a ColumnDataSource
sr_cds = ColumnDataSource(service_requests)
sr_cds_closed = ColumnDataSource(sr_closed)
sr_cds_open = ColumnDataSource(sr_open)

# create the blank figure & use webgl to use GPU rendering in browser
p = figure(webgl=True)

# add a map
p.add_tile(CARTODBPOSITRON)

# create circle glyphs with wm_y and wm_x coordinates as x and y
p.circle('wm_x', 'wm_y', source=sr_cds)

# create a menu widget to be added to the curdoc
menu = Select(options=['Closed', 'Open'], value='Closed', title='Case Status')

# create a function to connect the menu with the data source
def update_plot(attr, old, new):
	if menu.value == 'Closed':
		# take the original CDS and replace the values from the wm_y and wm_x keys
		# with the values from the filtered CDS
		sr_cds.data = {
		'wm_y':sr_cds_closed.data['wm_y'],
		'wm_x':sr_cds_closed.data['wm_x'],
		}
	else:
		sr_cds.data = {
		'wm_y':sr_cds_open.data['wm_y'],
		'wm_x':sr_cds_open.data['wm_x'],
		}


menu.on_change('value', update_plot)

# create a layout with one row
layout = row(p, menu)

# add the layout to the current document
curdoc().add_root(layout)